In [18]:
import torch
import hypothesis
import math

In [19]:
def find_common_reshape(init_a, init_b):
    a = list(init_a)
    b = list(init_b)

    while len(a) < len(b):
        a.insert(0, 1)
    while len(b) < len(a):
        b.insert(0, 1)

    N = len(a)
    a_res = 1
    b_res = 1
    for i in range(N - 1, -1, -1):
        if a[i] == 1 or b[i] == 1:
            continue
        else:
            a_res *= a[i]
            b_res *= b[i]
            gcd = math.gcd(a_res, b_res)
            a_res = int(a_res / gcd)
            b_res = int(b_res / gcd)
            a[i] = gcd
            b[i] = gcd

    if a_res != 1 or b_res != 1:
        a.insert(0, 1)
        b.insert(0, b_res)
        a.insert(0, a_res)

    return a, b

In [20]:
class DSU:
    def __init__(self, n: int):
        self.e = [-1] * n
        self.list_head = [torch.tensor(1)] * n

    def get(self, x: int) -> int:
        """Find with path compression."""
        if self.e[x] < 0:
            return x
        self.e[x] = self.get(self.e[x])
        return self.e[x]

    def get_head(self, x: int):
        return self.list_head[self.get(x)]

    def size(self, x: int) -> int:
        """Return size of the set containing x."""
        root = self.get(x)
        return -self.e[root]

    def unite(self, x: int, y: int, op) -> bool:
        """Union by size. Returns True if merged, False if already same set."""
        x = self.get(x)
        y = self.get(y)
        if x == y:
            return False
        if self.e[x] > self.e[y]:
            x, y = y, x
        self.e[x] += self.e[y]
        self.e[y] = x

        self.list_head[x] = op(self.list_head[x], self.list_head[y])
        self.list_head[y] = None
        return True

In [1]:
from hypothesis import given
from hypothesis.strategies import lists, integers

@given(lists(integers(min_value=1)))
def test_graph_builder(shapes):
    print(shapes)